In [1]:
#https://www.mlflow.org/docs/latest/tutorials-and-examples/tutorial.html
#https://pedro-munoz.tech/how-to-setup-mlflow-in-production/
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import git

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

sys.path.append('../')

In [2]:
#tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [4]:
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

In [6]:
def savefig(rmse, mae, r2, artifact_path):
    df = pd.DataFrame()
    df['mae'] = [mae.round(3)]
    df['rmse'] = [rmse.round(3)]
    df['r2'] = [r2.round(3)]

    fig,ax = render_mpl_table(df, header_columns=0, col_width=2.0)
    fig.savefig(artifact_path+"/metrics.png")  

In [7]:
def get_commit_version():
    repo = git.Repo(search_parent_directories=True)
    return repo.head.object.hexsha

In [9]:
warnings.filterwarnings("ignore")
np.random.seed(40)

data = pd.read_csv("data/winequality-red.csv", sep=";")

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [10]:
alpha = 0.2
l1_ratio = 1
run_name = 'Tutorial MLFlow - ML Lead'
description = 'Exemplo'
experiment_id = 3

In [11]:
artifact_path = "artifacts"
with mlflow.start_run(run_name=run_name, experiment_id = 3, description=description, tags={"mlflow.source.git.commit": get_commit_version()}):
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    test_x.to_csv(artifact_path+"/test.csv")
    train_x.to_csv(artifact_path+"/train.csv")

    savefig(rmse,mae,r2, artifact_path)

    mlflow.log_artifact(artifact_path+"/test.csv", artifact_path="data")
    mlflow.log_artifact(artifact_path+"/train.csv", artifact_path="data")
    mlflow.log_artifact(artifact_path+"/metrics.png", artifact_path="images")

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        # model_info = mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        model_info = mlflow.sklearn.log_model(lr, "model")
    else:
        mlflow.sklearn.log_model(lr, "model")

InvalidGitRepositoryError: 